In [31]:
pip install pyfinite

Note: you may need to restart the kernel to use updated packages.


In [32]:
import numpy as np
from numpy.linalg import matrix_rank
np.set_printoptions(threshold=np.inf)
from pyfinite import ffield
import random
import sympy as sp

dct = {'f' : [0,0,0,0],
 'g' : [0,0,0,1],
 'h' : [0,0,1,0],
 'i' : [0,0,1,1],
 'j' : [0,1,0,0],
 'k' : [0,1,0,1],
 'l' : [0,1,1,0],
 'm' : [0,1,1,1],
 'n' : [1,0,0,0],
 'o' : [1,0,0,1],
 'p' : [1,0,1,0],
 'q' : [1,0,1,1],
 'r' : [1,1,0,0],
 's' : [1,1,0,1],
 't' : [1,1,1,0],
 'u' : [1,1,1,1]}

inv_dict = {'0000': 'f',
 '0001': 'g',
 '0010': 'h',
 '0011': 'i',
 '0100': 'j',
 '0101': 'k',
 '0110': 'l',
 '0111': 'm',
 '1000': 'n',
 '1001': 'o',
 '1010': 'p',
 '1011': 'q',
 '1100': 'r',
 '1101': 's',
 '1110': 't',
 '1111': 'u'}

def byte_str(b):
    binnum = '{:0>8}'.format(format(b,"b"))
    a = inv_dict[binnum[0:4]], inv_dict[binnum[4:8]]
    return a[0]+a[1]

def map_to_str(st):
    char = chr(16*(ord(st[0]) - ord('f')) + ord(st[1]) - ord('f'))
    return char

def block_to_byte(c):
    plainText = ""
    for i in range(0, len(c), 2):
        plainText += map_to_str(c[i:i+2])
    return plainText

exp_store = [[-1]*128 for i in range(128)]

F = ffield.FField(7)

def Add (n1, n2):
    return int(n1) ^ int(n2)

def Multiply (n1, n2):
    return F.Multiply(n1, n2)

def Expo (no, pow):
    if exp_store[no][pow] != -1:
        return exp_store[no][pow]

    result = 0;
    if pow == 0:
        result = 1
    elif pow == 1:
        result = no
    elif pow%2 == 0:
        sqrt_no = Expo(no, pow>>1)
        result = Multiply(sqrt_no, sqrt_no)
    else:
        sqrt_no = Expo(no, pow>>1)
        result = Multiply(sqrt_no, sqrt_no)
        result = Multiply(no, result)

    exp_store[no][pow] = result
    return result

def addVectors (v1, v2):
    result = [0]*8
    for i, (e1, e2) in enumerate(zip(v1, v2)):
        result[i] = Add(e1, e2)
    return result

def scalarMultiplication (v, elem):
    result = [0]*8
    for i, e in enumerate(v):
        result[i] = Multiply(e, elem)
    return result

def LinearTransformation (mat, elist):
    result = [0]*8
    for row, elem in zip(mat, elist):
        result = addVectors(scalarMultiplication(row, elem), result)
    return result

poss_exp = [[] for i in range(8)]

poss_aii = [[[] for i in range(8)] for j in range(8)]
input_file = open("inputs.txt", 'r')
output_file = open("output1.txt", 'r')
for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    inpString = []
    outString = []
   
    for hexi in iline.strip().split(" "):
        inpString.append(block_to_byte(hexi)[ind])
    for hexo in oline.strip().split(" "):
        outString.append(block_to_byte(hexo)[ind])
        
    for i in range(1, 127):
        for j in range(1, 128):
            flag = True
            for inps, outs in zip(inpString, outString):
                
                if ord(outs) != Expo(Multiply(Expo(Multiply(Expo(ord(inps), i), j), i), j), i):
                    flag = False
                    break
            if flag:
              
                poss_exp[ind].append(i)
                poss_aii[ind][ind].append(j)
print(poss_aii)
print(poss_exp)

[[[84, 28, 113], [], [], [], [], [], [], []], [[], [86, 37, 70], [], [], [], [], [], []], [[], [], [105, 43, 107], [], [], [], [], []], [[], [], [], [68, 95, 12], [], [], [], []], [[], [], [], [], [118, 112, 20], [], [], []], [[], [], [], [], [], [11, 122, 100], [], []], [[], [], [], [], [], [], [16, 27, 28], []], [[], [], [], [], [], [], [], [38, 23, 31]]]
[[18, 21, 88], [33, 102, 119], [17, 41, 69], [17, 41, 69], [40, 89, 125], [45, 93, 116], [8, 25, 94], [16, 50, 61]]


In [33]:
input_file = open("inputs.txt", 'r')
output_file = open("output1.txt", 'r')
for ind, (iline, oline) in enumerate(zip(input_file.readlines(), output_file.readlines())):
    
    if ind > 6 :
        break
    inpString = []
    outString = []
    
    for hexi in iline.strip().split(" "):
        inpString.append(block_to_byte(hexi)[ind]) 
    for hexo in oline.strip().split(" "):
        outString.append(block_to_byte(hexo)[ind+1])
    for i in range(1, 128):
        
        for p1, e1 in zip(poss_exp[ind+1], poss_aii[ind+1][ind+1]):
            for p2, e2 in zip(poss_exp[ind], poss_aii[ind][ind]):
                flag = True
                for inp, outp in zip(inpString, outString):
                   
                    if ord(outp) != Expo(Add(Multiply(Expo(Multiply(Expo(ord(inp), p2), e2), p2), i) ,Multiply(Expo(Multiply(Expo(ord(inp), p2), i), p1), e1)), p1):
                        flag = False
                        break
                if flag:
                    
                    poss_exp[ind+1] = [p1]
                    poss_aii[ind+1][ind+1] = [e1]
                    poss_exp[ind] = [p2]
                    poss_aii[ind][ind] = [e2]
                    poss_aii[ind][ind+1] = [i]
print(poss_aii)
print(poss_exp)

[[[84], [124], [], [], [], [], [], []], [[], [70], [29], [], [], [], [], []], [[], [], [43], [1], [], [], [], []], [[], [], [], [12], [104], [], [], []], [[], [], [], [], [112], [101], [], []], [[], [], [], [], [], [11], [93], []], [[], [], [], [], [], [], [27], [24]], [[], [], [], [], [], [], [], [38]]]
[[18], [119], [41], [69], [89], [45], [25], [16]]


In [34]:

def EAEAE (plaintext, lin_mat, exp_mat):
    plaintext = [ord(c) for c in plaintext]
    Output = [[0 for j in range (8)] for i in range(8)]
    for ind, elem in enumerate(plaintext):
        Output[0][ind] = Expo(elem, exp_mat[ind])

    Output[1] = LinearTransformation(lin_mat, Output[0])

    for ind, elem in enumerate(Output[1]):
        Output[2][ind] = Expo(elem, exp_mat[ind])

    Output[3] = LinearTransformation(lin_mat, Output[2])

    for ind, elem in enumerate(Output[3]):
        Output[4][ind] = Expo(elem, exp_mat[ind])
    return Output[4]

for index in range(6):
   
    of = index + 2
    
    exp_list = [e[0] for e in poss_exp]
    lin_trans_list = [[0 for i in range(8)] for j in range(8)]
 
    for i in range(8):
        for j in range(8):
            lin_trans_list[i][j] = 0 if len(poss_aii[i][j]) == 0 else poss_aii[i][j][0]
    inp = open("inputs.txt", 'r')
    out = open("output1.txt", 'r')
    for ind, (iline, oline) in enumerate(zip(inp.readlines(), out.readlines())):
        if ind > (7-of):
            continue
        inpString = [block_to_byte(msg) for msg in iline.strip().split(" ")]
        outString = [block_to_byte(msg) for msg in oline.strip().split(" ")]
       
        for i in range(1, 128):
            lin_trans_list[ind][ind+of] = i
            flag = True
            for inps, outs in zip(inpString, outString):
                if EAEAE(inps, lin_trans_list, exp_list)[ind+of] != ord(outs[ind+of]):
                    flag = False
                    break
            if flag:
                poss_aii[ind][ind+of] = [i]
    inp.close();
    out.close();

lin_trans_list = [[0 for p in range(8)] for q in range(8)]
for p in range(8):
    for q in range(8):
        lin_trans_list[p][q] = 0 if len(poss_aii[p][q]) == 0 else poss_aii[p][q][0]

print(lin_trans_list)
print(exp_list)

[[84, 124, 9, 102, 100, 18, 22, 65], [0, 70, 29, 21, 36, 52, 119, 8], [0, 0, 43, 1, 3, 31, 19, 80], [0, 0, 0, 12, 104, 41, 100, 25], [0, 0, 0, 0, 112, 101, 25, 10], [0, 0, 0, 0, 0, 11, 93, 73], [0, 0, 0, 0, 0, 0, 27, 24], [0, 0, 0, 0, 0, 0, 0, 38]]
[18, 119, 41, 69, 89, 45, 25, 16]


In [35]:
A = [[84, 124, 9, 102, 100, 18, 22, 65], 
                 [0, 70, 29, 21, 36, 52, 119, 8], 
                 [0, 0, 43, 1, 3, 31, 19, 80], 
                 [0, 0, 0, 12, 104, 41, 100, 25], 
                 [0, 0, 0, 0, 112, 101, 25, 10], 
                 [0, 0, 0, 0, 0, 11, 93, 73], 
                 [0, 0, 0, 0, 0, 0, 27, 24], 
                 [0, 0, 0, 0, 0, 0, 0, 38]]
E = [18, 119, 41, 69, 89, 45, 25, 16]


pass_1 = "ijmimhmqhhfqiigt"
pass_2 = "kkftkfhfktlokmhp"


def DecryptPassword(password):
    passw = block_to_byte(password)
    op = ""
    for ind in range(8):
        for ans in range(128):
            inp = op + byte_str(ans)+(16-len(op)-2)*'f'
            if ord(passw[ind]) == EAEAE(block_to_byte(inp), A, E)[ind]:
                op += byte_str(ans)
                break
    return op

print(block_to_byte(DecryptPassword(pass_1))+block_to_byte(DecryptPassword(pass_2)))


rxrkarrmdo000000
